# 사전 구축된 ReAct 에이전트에서 구조화된 출력을 반환하는 방법

!!! 정보 "필수 조건"
    이 가이드는 다음에 대한 친숙함을 가정합니다:
    
    - [에이전트 아키텍처](../../concepts/agentic_concepts/)
    - [채팅 모델](https://python.langchain.com/docs/concepts/chat_models/)
    - [도구](https://python.langchain.com/docs/concepts/tools/)
    - [구조화된 출력](https://python.langchain.com/docs/concepts/structured_outputs/)

사전 구축된 ReAct 에이전트에서 구조화된 출력을 반환하려면 원하는 출력 스키마와 함께 `response_format` 매개변수를 [create_react_agent][langgraph.prebuilt.chat_agent_executor.create_react_agent]에 제공할 수 있습니다:

```python
class ResponseFormat(BaseModel):
    """사용자에게 이 형식으로 응답합니다."""
    my_special_output: str


graph = create_react_agent(
    model,
    tools=tools,
    # `response_format` 매개변수를 사용하여 구조화된 출력의 스키마를 지정합니다.
    response_format=ResponseFormat
)
```

사전 구축된 ReAct는 ReAct 루프의 끝에서 구조화된 출력 응답을 생성하기 위해 추가적인 LLM 호출을 수행합니다. 도구 호출 에이전트에서 구조화된 출력을 반환하는 다른 전략에 대해서는 [이 가이드](../react-agent-structured-output)를 참조하십시오.


## 설정

먼저 필요한 패키지를 설치하고 API 키를 설정합시다.


In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai


In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위해 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 빠르게 발견하고 성능을 향상시키세요. LangSmith는 LangGraph로 구축된 LLM 앱을 디버깅하고 테스트하며 모니터링하기 위해 추적 데이터를 사용할 수 있게 해줍니다 — 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>를 참조하세요.
    </p>
</div>


## 코드


In [3]:
# First we initialize the model we want to use.
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)

from typing import Literal
from langchain_core.tools import tool


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]

# Define the structured output schema

from pydantic import BaseModel, Field


class WeatherResponse(BaseModel):
    """Respond to the user in this format."""

    conditions: str = Field(description="Weather conditions")


# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(
    model,
    tools=tools,
    # specify the schema for the structured output using `response_format` parameter
    response_format=WeatherResponse,
)


## 사용법

이제 우리의 에이전트를 테스트해 보겠습니다:


In [4]:
inputs = {"messages": [("user", "What's the weather in NYC?")]}
response = graph.invoke(inputs)


에이전트 출력에는 지정된 `WeatherResponse` 스키마에 맞는 구조화된 출력이 포함된 `structured_response` 키와 `messages` 키 아래의 메시지 기록이 포함되어 있음을 알 수 있습니다.


In [5]:
response["structured_response"]


WeatherResponse(conditions='cloudy')

### 프롬프트 사용자 정의


두 번째 LLM 호출을 구조화된 출력 생성을 위해 추가로 사용자화할 필요가 있을 수 있으며, 시스템 프롬프트를 제공해야 합니다. 이를 위해 튜플(prompt, schema)을 전달할 수 있습니다.


In [6]:
graph = create_react_agent(
    model,
    tools=tools,
    # specify both the system prompt and the schema for the structured output
    response_format=("Always return capitalized weather conditions", WeatherResponse),
)

inputs = {"messages": [("user", "What's the weather in NYC?")]}
response = graph.invoke(inputs)


구조화된 응답에 이제 대문자 값이 포함되어 있음을 확인할 수 있습니다:


In [7]:
response["structured_response"]


WeatherResponse(conditions='Cloudy')